In [1]:
import pandas as pd

column_names =['userId', 'sys', 'segId', 'Q', 'src', 'tgt', 'score', 'docId', 'whole_doc', 'start', 'end']
wmt21 = pd.read_csv('wmt21.csv', names=column_names, header=None)

In [3]:
en_cs = wmt21[(wmt21['Q']=='TGT')&(wmt21['src']=='eng')&(wmt21['tgt']=='ces')]
en_cs.to_csv('eng_ces21.csv')

en_ru = wmt21[(wmt21['Q']=='TGT')&(wmt21['src']=='eng')&(wmt21['tgt']=='rus')]
en_ru.to_csv('eng_rus21.csv')

In [5]:
df = pd.read_csv('eng_ces21.csv')

In [6]:
filtered = df[df['score']>=98]
filtered.to_csv('en_cs_21_98.csv')

In [8]:
meta = pd.read_csv('metadata/newstest2021.en-cs.csv', sep='\t')
meta.head()

,segment,document
0,1,en.ndtv.com.75164
1,2,en.ndtv.com.75164
2,3,en.ndtv.com.75164
3,4,en.ndtv.com.75164
4,5,en.ndtv.com.75164


In [13]:
meta['num'] = range(1, len(meta) + 1)

In [9]:
systems = filtered['sys'].unique()

In [11]:
import os

if not os.path.isdir('best-lines-21'):
        os.mkdir('best-lines-21')

In [19]:
all_best_lines = open('best-lines-21/all-best-lines-cs-21.txt', 'w')

In [22]:
filtered['segId'] = filtered['segId'].astype(str)

C:\Users\10056409\AppData\Local\Temp\ipykernel_13436\1698710042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['segId'] = filtered['segId'].astype(str)


In [23]:
for sys in systems:
    temp = filtered[filtered['sys']==sys]
    docs = temp['docId'].unique()
    doc_sent = {}
    for doc in docs:
        lines = set()
        doc_lines = temp.loc[temp['docId'] == doc, 'segId']
        doc_lines.columns =['n', 'idx']
        doc_lines = doc_lines.to_frame()
        for index, row in doc_lines.iterrows():
            lines.add(row['segId'])
        doc_sent[doc] = lines

    for key in doc_sent.keys():
        doc_sent[key] = list(doc_sent[key])
    
    valid_sentences = []
    for key in doc_sent.keys():
        for item in doc_sent[key]:
            if not '-' in item and item != '0':
                line = meta.loc[(meta['document'] == key) & (meta['segment'] == int(item))]['num']
                valid_sentences.append(line._values[0])
    sorted(valid_sentences)
    
    for sent in sorted(valid_sentences):
        sent = str(sent) + ',' + sys
        all_best_lines.write("%s\n" % sent)
        
all_best_lines.close()


In [29]:
import random

mt_final_dataset_lines = []
human_final_dataset_lines = []
all_best_lines = open('best-lines-21/all-best-lines-cs-21.txt', 'r')
lines = all_best_lines.readlines()
for i in range(1,1003):
    line_choice_mt = []
    line_choice_human = []
    for line in lines:
        line = line.strip()
        line_parts = line.split(',')
        if line_parts[0] == str(i):
            if line_parts[1] == 'translator-B':
                continue
            elif line_parts[1] == 'translator-A':
                line_choice_human.append(line)
            else:
                line_choice_mt.append(line)
    if len(line_choice_mt) != 0:
        rand_sent = random.choice(line_choice_mt)
        mt_final_dataset_lines.append(rand_sent)
    
    if len(line_choice_human) != 0:
        rand_sent = random.choice(line_choice_human)
        human_final_dataset_lines.append(rand_sent)
        
print(len(mt_final_dataset_lines))
print(len(human_final_dataset_lines))

992
792


In [30]:
sys_sents_mt = {}
for line in mt_final_dataset_lines:
    line = line.strip()
    line_parts = line.split(',')
    if line_parts[1] in sys_sents_mt.keys():
        sys_sents_mt[line_parts[1]].append(int(line_parts[0]))
    else:
        sys_sents_mt[line_parts[1]] = [int(line_parts[0])]

sys_sents_human = {}
for line in human_final_dataset_lines:
    line = line.strip()
    line_parts = line.split(',')
    if line_parts[1] in sys_sents_human.keys():
        sys_sents_human[line_parts[1]].append(int(line_parts[0]))
    else:
        sys_sents_human[line_parts[1]] = [int(line_parts[0])]

In [32]:
final_mt_dataset = open('best-lines-21/mt-dataset-21-cs.txt', 'w', encoding='UTF-8')
final_human_dataset = open('best-lines-21/human-dataset-21-cs.txt', 'w', encoding='UTF-8')


for sys in sys_sents_mt.keys():
    sys_output = open(f'en-cs-21/newstest2021.en-cs.hyp.{sys}.cs.txt', 'r', encoding='UTF-8')
    sents = sys_output.readlines()
    for num in sys_sents_mt[sys]:
        sent = sents[num-1].strip()
        sent = sent + '|mt'
        final_mt_dataset.write("%s\n" % sent)

for sys in sys_sents_human.keys():
    if sys == 'translator-A':
        sys_name = 'ref-A'
    sys_output = open(f'en-cs-21/newstest2021.en-cs.ref.{sys_name}.cs.txt', 'r', encoding='UTF-8')
    sents = sys_output.readlines()
    for num in sys_sents_human[sys]:
        sent = sents[num-1].strip()
        sent = sent + '|human'
        final_human_dataset.write("%s\n" % sent)
        
final_mt_dataset.close()
final_human_dataset.close()